# Part 2, Topic 1: CPA Attack on 32bit AES (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *So far, we've been focusing on a single implementation of AES, TINYAES128C (or AVRCRYPTOLIB, if you're on XMEGA). TINYAES128C, which is designed to run on a variety of microcontrollers, doesn't make any implementation specific optimizations. In this lab, we'll look at how we can break a 32-bit optimized version of AES using a CPA attack.*

**LEARNING OUTCOMES:**

* Understanding how AES can be optimized on 32-bit platforms.
* Attacking an optimized version of AES using CPA

## Optimizing AES

A 32-bit machine can operate on 32-bit words, so it seems wasteful to use the same 8-bit operations. For example, if we look at the SBox operation:

$$
b = sbox(state) = sbox(\left[ \begin{array}
& S0 & S4 & S8 & S12 \\
S1 & S5 & S9 & S13 \\
S2 & S6 & S10 & S14 \\
S3 & S7 & S11 & S15
\end{array} \right]) = \left[ \begin{array}
& S0 & S4 & S8 & S12 \\
S5 & S9 & S13 & S1 \\
S10 & S14 & S2 & S6 \\
S15 & S3 & S7 & S11
\end{array} \right]
$$

we could consider each row as a 32-bit number and do three bitwise rotates instead of moving a bunch of stuff around in memory. Even better, we can speed up AES considerably by generating 32-bit lookup tables, called T-Tables, as was described in the book [The Design of Rijndael](http://www.springer.com/gp/book/9783540425809) which was published by the authors of AES.

In order to take full advantage of our 32 bit machine, we can examine a typical round of AES. With the exception of the final round, each round looks like:

$\text{a = Round Input}$

$\text{b = SubBytes(a)}$

$\text{c = ShiftRows(b)}$

$\text{d = MixColumns(c)}$

$\text{a' = AddRoundKey(d) = Round Output}$

We'll leave AddRoundKey the way it is. The other operations are:

$b_{i,j} = \text{sbox}[a_{i,j}]$

$\left[ \begin{array} { c } { c _ { 0 , j } } \\ { c _ { 1 , j } } \\ { c _ { 2 , j } } \\ { c _ { 3 , j } } \end{array} \right] = \left[ \begin{array} { l } { b _ { 0 , j + 0 } } \\ { b _ { 1 , j + 1 } } \\ { b _ { 2 , j + 2 } } \\ { b _ { 3 , j + 3 } } \end{array} \right]$

$\left[ \begin{array} { l } { d _ { 0 , j } } \\ { d _ { 1 , j } } \\ { d _ { 2 , j } } \\ { d _ { 3 , j } } \end{array} \right] = \left[ \begin{array} { l l l l } { 02 } & { 03 } & { 01 } & { 01 } \\ { 01 } & { 02 } & { 03 } & { 01 } \\ { 01 } & { 01 } & { 02 } & { 03 } \\ { 03 } & { 01 } & { 01 } & { 02 } \end{array} \right] \times \left[ \begin{array} { c } { c _ { 0 , j } } \\ { c _ { 1 , j } } \\ { c _ { 2 , j } } \\ { c _ { 3 , j } } \end{array} \right]$

Note that the ShiftRows operation $b_{i, j+c}$ is a cyclic shift and the matrix multiplcation in MixColumns denotes the xtime operation in GF($2^8$).

It's possible to combine all three of these operations into a single line. We can write 4 bytes of $d$ as the linear combination of four different 4 byte vectors:

$\left[ \begin{array} { l } { d _ { 0 , j } } \\ { d _ { 1 , j } } \\ { d _ { 2 , j } } \\ { d _ { 3 , j } } \end{array} \right] = \left[ \begin{array} { l } { 02 } \\ { 01 } \\ { 01 } \\ { 03 } \end{array} \right] \operatorname { sbox } \left[ a _ { 0 , j + 0 } \right] \oplus \left[ \begin{array} { l } { 03 } \\ { 02 } \\ { 01 } \\ { 01 } \end{array} \right] \operatorname { sbox } \left[ a _ { 1 , j + 1 } \right] \oplus \left[ \begin{array} { c } { 01 } \\ { 03 } \\ { 02 } \\ { 01 } \end{array} \right] \operatorname { sbox } \left[ a _ { 2 , j + 2 } \right] \oplus \left[ \begin{array} { c } { 01 } \\ { 01 } \\ { 03 } \\ { 02 } \end{array} \right] \operatorname { sbox } \left[ a _ { 3 , j + 3 } \right]$

Now, for each of these four components, we can tabulate the outputs for every possible 8-bit input:

$T _ { 0 } [ a ] = \left[ \begin{array} { l l } { 02 \times \operatorname { sbox } [ a ] } \\ { 01 \times \operatorname { sbox } [ a ] } \\ { 01 \times \operatorname { sbox } [ a ] } \\ { 03 \times \operatorname { sbox } [ a ] } \end{array} \right]$

$T _ { 1 } [ a ] = \left[ \begin{array} { l } { 03 \times \operatorname { sbox } [ a ] } \\ { 02 \times \operatorname { sbox } [ a ] } \\ { 01 \times \operatorname { sbox } [ a ] } \\ { 01 \times \operatorname { sbox } [ a ] } \end{array} \right]$

$T _ { 2 } [ a ] = \left[ \begin{array} { l l } { 01 \times \operatorname { sbox } [ a ] } \\ { 03 \times \operatorname { sbox } [ a ] } \\ { 02 \times \operatorname { sbox } [ a ] } \\ { 01 \times \operatorname { sbox } [ a ] } \end{array} \right]$

$T _ { 3 } [ a ] = \left[ \begin{array} { l l } { 01 \times \operatorname { sbox } [ a ] } \\ { 01 \times \operatorname { sbox } [ a ] } \\ { 03 \times \operatorname { sbox } [ a ] } \\ { 02 \times \operatorname { sbox } [ a ] } \end{array} \right]$

These tables have 2^8 different 32-bit entries, so together the tables take up 4 kB. Finally, we can quickly compute one round of AES by calculating

$\left[ \begin{array} { l } { d _ { 0 , j } } \\ { d _ { 1 , j } } \\ { d _ { 2 , j } } \\ { d _ { 3 , j } } \end{array} \right] = T _ { 0 } \left[ a _ { 0 } , j + 0 \right] \oplus T _ { 1 } \left[ a _ { 1 } , j + 1 \right] \oplus T _ { 2 } \left[ a _ { 2 } , j + 2 \right] \oplus T _ { 3 } \left[ a _ { 3 } , j + 3 \right]$

All together, with AddRoundKey at the end, a single round now takes 16 table lookups and 16 32-bit XOR operations. This arrangement is much more efficient than the traditional 8-bit implementation. There are a few more tradeoffs that can be made: for instance, the tables only differ by 8-bit shifts, so it's also possible to store only 1 kB of lookup tables at the expense of a few rotate operations.

While the TINYAES128C library we've been using doesn't make this optimization, another library included with ChipWhisperer called MBEDTLS does.

In [ ]:
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET = 'MBEDTLS'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex'.format(PLATFORM)
cw.program_target(scope, prog, fw_path)

In [ ]:
#Capture Traces
from tqdm import tnrange, trange
import numpy as np
import time

ktp = cw.ktp.Basic()

traces = []
N = 50000  # Number of traces
project = cw.create_project("32bit_AES.cwp", overwrite=True)

for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here

    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)

print(scope.adc.trig_count)

If we plot the AES power trace:

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

plt.figure()
plt.plot(project.waves[0])
plt.show()

You probably can't even pick out the different AES rounds anymore (whereas it was pretty obvious on TINYAES128C). MBED is also way faster - we only got part way into round 2 with 5000 samples of TINYAES, but with MBED we can finish the entire encryption in less than 5000 samples! Two questions we need to answer now are:

1. Is it possible for us to break this AES implementation?
1. If so, what sort of leakage model do we need?

As it turns out, the answers are:

1. Yes!
1. We can continue to use the same leakage model - the SBox output

This might come as a surprise, but it's true! Shift rows is just moving things around, so it doesn't really change anything about our attack, and multiplication step for MixColumns is linear, meaning it doesn't actually affect the attack either:

In [ ]:
@scared.attack_selection_function
def sbox_output(plaintext, guesses):
    res = np.empty((plaintext.shape[0], len(guesses), plaintext.shape[1]), dtype='uint8')
    for i, guess in enumerate(guesses):
        res[:,i,:] = scared.aes.sub_bytes(np.bitwise_xor(plaintext, guess))
    return res

In [ ]:
a = scared.CPAAttack(selection_function=sbox_output,
                    model=scared.HammingWeight(),
                    discriminant=scared.maxabs)

In [ ]:
import estraces
cw_traces = estraces.read_ths_from_ram(np.array([wave for wave in project.waves], dtype='float64'), 
                                      plaintext=np.array([textin for textin in project.textins], dtype='uint8'))

In [ ]:
%%time
container = scared.Container(cw_traces)
a.run(container)
for byte in np.argmax(a.scores, axis=0):
    print("{:02X}".format(byte%0xFF))

In [ ]:
len(a.results)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
p = figure()
key_guess = []
for i in range(16):
    results = a.results
    sorted_scores = np.argsort(a.scores, axis=0)
    guess = sorted_scores[-1][i]
    sec_guess = sorted_scores[-2][i]
    third_guess = sorted_scores[-3][i]
    xrange = range(len(results[guess][i]))
    print("Best Guess is {:02X} (Corr = {})".format(guess, a.scores[guess][i]))
    p.line(xrange, results[guess][i], color="red")
    p.line(xrange, results[sec_guess][i], color="green")
    p.line(xrange, results[third_guess][i], color="green")
    key_guess.append(guess)
    
show(p)

In [ ]:
np.argmax(a.scores.transpose()[0][:])

In [ ]:
np.sort(a.scores.transpose()[0][:])

In [ ]:
import scared

def ahhh(ciphertext, guesses):
    res = np.empty((ciphertext.shape[0], len(guesses), ciphertext.shape[1]), dtype='uint8')
    for i, guess in enumerate(guesses):
        st10 = scared.aes.shift_rows(ciphertext)
        st9 = scared.aes.inv_sub_bytes(np.bitwise_xor(ciphertext, guess))
        res[:,i,:] = np.bitwise_xor(st9, st10)
    return res

a = scared.CPAAttack(selection_function=ahhh,
                    model=scared.HammingWeight(),
                    discriminant=scared.maxabs)
asdfasdf = estraces.read_ths_from_ram(np.array([wave for wave in project.waves], dtype='float64'), 
                                      ciphertext=np.array([textin for textin in project.textouts], dtype='uint8'))

container = scared.Container(asdfasdf)
a.run(container)
print(a.scores.transpose()[0][0xD0])
print(a.scores.transpose()[1][0x14])
for byte in np.argmax(a.scores, axis=0):
    print("{:02X}".format(byte%0xFF))

In [ ]:
import scared

a = scared.CPAAttack(selection_function=ahhh,
                    model=scared.HammingWeight(),
                    discriminant=scared.maxabs)
asdfasdf = estraces.read_ths_from_ram(np.array([wave for wave in project.waves], dtype='float64'), 
                                      plaintext=np.array([textin for textin in project.textins], dtype='uint8'),
                                      ciphertext=np.array([textout for textout in project.textouts], dtype='uint8'))